**Supervised Machine Learning Homework - Predicting Credit Risk**


In [3]:
#import dependencies
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [4]:
#load the files
train_df = pd.read_csv(Path('Resources/2019loans.csv'))
test_df = pd.read_csv(Path('Resources/2020Q1loans.csv'))
# train_df.head()
test_df.head(4)

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,target
0,26400.0,0.1033,565.22,MORTGAGE,70000.0,Not Verified,n,24.48,0.0,0.0,...,0.0,0.0,0.0,258000.0,38985.0,48100.0,30000.0,N,N,low_risk
1,25000.0,0.0819,785.61,MORTGAGE,100000.0,Verified,n,32.76,0.0,2.0,...,0.0,0.0,0.0,427746.0,97284.0,51200.0,116246.0,N,N,low_risk
2,10200.0,0.1430,350.10,MORTGAGE,40000.0,Not Verified,n,20.67,0.0,0.0,...,100.0,0.0,0.0,28204.0,9242.0,2500.0,24204.0,N,N,low_risk
3,15000.0,0.2305,423.29,MORTGAGE,58500.0,Not Verified,n,18.99,0.0,2.0,...,25.0,0.0,0.0,44700.0,30868.0,20000.0,23000.0,N,N,low_risk


**Preprocessing the Data**


In [5]:
# Separate the target column (y) from the predictive features (X) in both our train and test dfs & make a copy of its data
X_train = train_df.drop("target", axis=1).copy()
X_test = test_df.drop("target", axis=1).copy()
y_train = train_df["target"].copy()
y_test = test_df["target"].copy()

In [6]:
# Convert categorical data to numeric and separate target feature for training data
X_train = pd.get_dummies(train_df.drop(columns = ['target']))
X_train.head()
X_test = pd.get_dummies(test_df.drop(columns = ['target']))
X_test.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,verification_status_Source Verified,verification_status_Verified,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,hardship_flag_Y,debt_settlement_flag_N
0,26400.0,0.1033,565.22,70000.0,24.48,0.0,0.0,10.0,0.0,20070.0,...,0,0,1,0,1,1,0,1,0,1
1,25000.0,0.0819,785.61,100000.0,32.76,0.0,2.0,13.0,0.0,8168.0,...,0,1,1,0,1,0,1,1,0,1
2,10200.0,0.1430,350.10,40000.0,20.67,0.0,0.0,8.0,0.0,3306.0,...,0,0,1,0,1,1,0,1,0,1
3,15000.0,0.2305,423.29,58500.0,18.99,0.0,2.0,9.0,0.0,12477.0,...,0,0,1,0,1,1,0,1,0,1
4,40000.0,0.1102,870.10,90000.0,38.87,0.0,2.0,15.0,0.0,24111.0,...,0,0,1,0,1,1,0,1,0,1


In [7]:
# First, add dummy variables that are missing in the testing set
cols_to_add = set(X_train.columns) ^ set(X_test.columns)
# Loop through each missing column and add it to X_test with all 0s
for col in cols_to_add:
        X_test[col] = 0
        
# Set the order of the columns in X_test to match the order of columns in X_train
X_test = X_test[X_train.columns]
# Encode the target column with 1s and 0s (both train and test)
X_test = X_test[X_train.columns]
target_encoder = LabelEncoder().fit(y_train)
y_train = target_encoder.transform(y_train)
y_test=target_encoder.transform(y_test)
y_test

array([1, 1, 1, ..., 0, 0, 0])

In [8]:
# Check the shapes to make sure X_train and X_test have the same number of columns (same with y_train/y_test)
# Check that X_train and y_train have the same number of rows (same with X_test/y_test)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(12180, 92)
(8146, 92)
(12180,)
(8146,)


***Predictions***

Using an entire year's worth of data (2019), my task is to predict the credit risk of loans from the first quarter of the next year (2020). The data is undersampled to give an even number of high risk and low risk loans. In the original dataset, only 2.2% of loans are categorized as high risk. This analysis is trying to find the best model that can detect if a loan is high risk or not. To come up with a solutiom, I will be using the following models: a LogisticRegression and RandomForestClassifier. The logistic regression will be a better suited model for this credit risk case, which has relatively direct data. The Logistic regression does well on highly correlated data which is true in this case. The random forest fits a number of decision tree classifiers on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting. But in this case, using the random forest will be overfitting and logistic will be more fitted since the credit risk data that I was provided with is strongly linked together/highly correlated.

### logistic regression

In [20]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='lbfgs', random_state=24)
lr.fit(X_train, y_train)
print(lr.score(X_test, y_test))
print(lr.score(X_train, y_train))
# # Logistic regression is coming out with 50% which is bad

0.5040510680088387
0.6535303776683087


c:\users\aconnolly\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### RandomForestClassifier model

In [23]:
# Train a Random Forest Classifier model and print the model train and test score
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=35, max_depth=3)
rf.fit(X_train, y_train)
print(rf.score(X_test, y_test))
print(rf.score(X_train, y_train))

0.6977657746133071
0.722167487684729


In [15]:
# Scale the data: we need to scale only the X_train data only that one time
scaler = StandardScaler().fit(X_train)

# Use the scaler on X_train and X_test
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# We need to fit preprocessors to training data only! If we fit preprocessors before splitting the data, we're biasing the model with information
# from the testing set.

In [22]:
# Train the Logistic Regression model on the scaled data and print the model score
lr.fit(X_train_scaled, y_train)
print(lr.score(X_test_scaled, y_test))
print(lr.score(X_train_scaled, y_train))

#this way we prevented overfitting

0.8062852933955316
0.710919540229885


c:\users\aconnolly\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [24]:
# Train a Random Forest Classifier model on the scaled data and print the model train/test score
rf.fit(X_train_scaled, y_train)
print(rf.score(X_test_scaled, y_test))
print(rf.score(X_train_scaled, y_train))

rf.rf.feature_importances_

0.6919960716916278
0.7269293924466338


AttributeError: 'RandomForestClassifier' object has no attribute 'rf'